In [ ]:
import os
STEPS = "./steps"
if not os.path.isdir(STEPS):
    os.mkdir(STEPS)

# Download images

## Segment all images

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

# 1. Load the best model
# The .pt file will be downloaded automatically on the first run.
model = YOLO('yolo11x-seg.pt')

# 2. Load your image
image_path = "20250926_142509.jpg" # <-- Change to your image file
original_image = cv2.imread(image_path)

# 3. Run prediction
results = model(original_image, device='0')

# The result object contains all the segmentation data
result = results[0]
# Check if any masks were detected
if result.masks is not None:
    # 4. Loop through each detected object
    for i, mask in enumerate(result.masks.data):

        # Get the class name
        class_id = int(result.boxes.cls[i])
        class_name = model.names[class_id]

        print(f"Extracting object {i+1}: {class_name}")

        # Convert mask to a binary image (0 or 255) and resize to original image size
        binary_mask = mask.cpu().numpy().astype(np.uint8) * 255
        binary_mask = cv2.resize(binary_mask, (original_image.shape[1], original_image.shape[0]))
        
        # 5. Create a transparent RGBA image for the extracted object
        extracted_rgba = cv2.cvtColor(original_image, cv2.COLOR_BGR2BGRA)
        extracted_rgba[:, :, 3] = binary_mask # Set the alpha channel
        
        # Save the extracted object
        output_filename = f"extracted_{class_name}_{i+1}.png"
        cv2.imwrite(output_filename, extracted_rgba)

    print(f"\nSuccessfully extracted {len(result.masks.data)} objects.")
else:
    print("No objects were segmented in the image.")

WARNING yolo_nas_l.pt appears to require 'super_gradients', which is not in Ultralytics requirements.
AutoInstall will run now for 'super_gradients' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolo11n.pt'
requirements: Ultralytics requirement ['super_gradients'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "super_gradients" ' returned non-zero exit status 1.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "super_gradients" ' returned non-zero exit status 1.
WARNING requirements:  Command 'pip install --no-cache-dir "super_gradients" ' returned non-zero exit status 1.


ModuleNotFoundError: No module named 'super_gradients'

In [2]:
from ultralytics import YOLO
from PIL import Image

# 1. Load a pre-trained segmentation model
# The 'yolov8n-seg.pt' is a small and fast segmentation model.
# Other options include yolov8s-seg.pt, yolov8m-seg.pt, etc.
model = YOLO('yolo11x-seg.pt')

# 2. Define the image you want to process
source_image = 'https://ultralytics.com/images/bus.jpg' # You can use a URL or a local path

# 3. Run inference with hardware acceleration
# Specify device='cuda' for NVIDIA GPUs or device='mps' for Apple Silicon.
# You can also use a specific GPU like device=0.
results = model.predict(source="image.jpg", device='cuda', save=True)

# 4. Process results (optional)
# The 'results' object contains all the information.
# For segmentation, you can access the masks.
if results[0].masks:
    print(f"Found {len(results[0].masks)} objects with masks.")
    # You can access mask data via results[0].masks.data
else:
    print("No segmentation masks were found.")

# The predicted image with masks will be saved in a 'runs/segment/predict' directory.
print("\nSegmentation complete! Check the 'runs/segment/predict' folder for the output image.")


image 1/1 c:\FastProjects\DeepLearningRepo\image.jpg: 640x480 1 apple, 1 dining table, 36.9ms
Speed: 3.1ms preprocess, 36.9ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\FastProjects\DeepLearningRepo\runs\segment\predict
Found 2 objects with masks.

Segmentation complete! Check the 'runs/segment/predict' folder for the output image.
